##### Copyright 2019 The TensorFlow Neural Structured Learning Authors

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Regularización adversativa para clasificación de imágenes

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/neural_structured_learning/tutorials/adversarial_keras_cnn_mnist"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/neural_structured_learning/tutorials/adversarial_keras_cnn_mnist.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/neural_structured_learning/tutorials/adversarial_keras_cnn_mnist.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/neural_structured_learning/tutorials/adversarial_keras_cnn_mnist.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a>
</td>
</table>

## Descripción general

En este tutorial, exploraremos el uso del aprendizaje adversativo ([Goodfellow et al., 2014](https://arxiv.org/abs/1412.6572)) para clasificación de imágenes con aprendizaje estructurado neuronal (NSL).

La idea principal del aprendizaje adversativo es entrenar un modelo con datos perturbados en forma adversativa (también llamados ejemplos adversativos), además de los datos de entrenamiento orgánico. Para el ojo humano, estos ejemplos adversativos se ven iguales al original, pero la perturbación hará que el modelo se confunda y haga predicciones o clasificaciones incorrectas. Los ejemplos adversativos se construyen para engañar intencionalmente al modelo, para que haga predicciones o clasificaciones erróneas. Al entrenar dichos ejemplos, el modelo aprende a enfrentar con robustez la perturbación adversativa al hacer las predicciones.

En este tutorial, ilustramos el siguiente procedimiento de aplicación de aprendizaje adversativo para obtener modelos robustos con aprendizaje estructurado neuronal.

1. Creamos una red neuronal como modelo base. En este tutorial, el modelo base se crea con la API funcional `tf.keras`; este procedimiento es compatible con los modelos creados también con las API de aplicación de subclases y secuenciales `tf.keras`. Para más información, sobre los modelos Keras en TensorFlow, consulte esta [documentación](https://www.tensorflow.org/api_docs/python/tf/keras/Model).
2. Encapsulamos el modelo de base con la clase del encapsulador **`AdversarialRegularization`**, provista por el NSL, para crear una nueva instancia `tf.keras.Model`. El modelo nuevo incluirá una pérdida adversativa como un término de regularización en su objetivo de entrenamiento.
3. Convertimos ejemplos de los datos de entrenamiento en diccionarios de características.
4. Entrenamos y evaluamos el modelo nuevo.

## Repaso para principiantes

Hay una [explicación en video](https://youtu.be/Js2WJkhdU7k) alusiva al aprendizaje adversativo para la clasificación de imágenes, parte de la serie de YouTube sobre aprendizaje estructurado neuronal con TensorFlow. A continuación, hemos resumido los conceptos clave explicados en el video y ampliamos la explicación brindada en la sección anterior de la descripción general.

El NSL optimiza, conjuntamente, tanto las características como las señales estructuradas de las imágenes, para ayudar a las redes a aprender mejor. Pero ¿qué pasaría si no hubiese una estructura explícita disponible para entrenar a la red neuronal? En este tutorial se explica una opción que incluye la creación de vecinos adversativos (modificada a partir de la muestra original) para construir dinámicamente una estructura.

En primer lugar, los vecinos adversativos se definen como versiones modificadas de la imagen de muestra aplicadas con pequeñas perturbaciones que engañan a la red neuronal para que emita clasificaciones imprecisas. Estas perturbaciones, que han sido minuciosamente diseñadas, por lo común, se basan en la dirección del gradiente inversa y se proponen confundir a la red neuronal durante el entrenamiento. Los humanos no podríamos notar la diferencia entre una imagen de muestra y el vecino adversativo generado. Sin embargo, para la red neuronal, las perturbaciones aplicadas son efectivas para llevar a una conclusión inexacta.

Los vecinos adversativos generados, entonces, quedan conectados a la muestra. Por lo tanto, construyen dinámicamente una estructura arista a arista. Con esta conexión, las redes neuronales aprenden a mantener las similitudes entre la muestra y los vecinos adversativos. A la vez, evitan la confusión resultante de los errores en la clasificación. De este modo, mejoran la calidad y la exactitud de la red neuronal en general.

En el siguiente segmento de código se presenta una explicación de alto nivel de los pasos involucrados. En el resto de este tutorial se hace un análisis más profundo y con más detalles técnicos.

1. Lea y prepare los datos. Cargue el conjunto de datos y normalice las valores de las características para mantenerlos dentro del rango de [0,1].

```
import neural_structured_learning as nsl

(x_train, y_train), (x_train, y_train) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
```

1. Cree la red neuronal. Para este ejemplo se utilizó un modelo de base Keras secuencial.

```
model = tf.keras.Sequential(...)
```


1. Configure el modelo adversativo. Incluya los hiperparámetros:  el multiplicador aplicado a la regularización adversativa, los valores de diferencia elegidos empíricamente para la velocidad de aprendizaje o el tamaño del paso. Invoque la regularización adversativa con una clase encapsuladora en torno a la red neuronal construida.

```
adv_config = nsl.configs.make_adv_reg_config(multiplier=0.2, adv_step_size=0.05)
adv_model = nsl.keras.AdversarialRegularization(model, adv_config)
```

1. Concluya con un flujo de trabajo Keras estándar: compile, ajuste y evalúe.

```
adv_model.compile(optimizer='adam', loss='sparse_categorizal_crossentropy', metrics=['accuracy'])
adv_model.fit({'feature': x_train, 'label': y_train}, epochs=5)
adv_model.evaluate({'feature': x_test, 'label': y_test})
```

Lo que ve es un aprendizaje adversativo habilitado en dos pasos y en tres simples líneas de código. Es la simplicidad del marco de aprendizaje estructurado neuronal. En la siguientes secciones, ampliaremos a partir de este procedimiento.

## Preparación

Instalamos el paquete de aprendizaje estructurado neuronal.

In [ ]:
!pip install --quiet neural-structured-learning

Importamos las bibliotecas. Abreviamos `neural_structured_learning` con `nsl`.

In [ ]:
import matplotlib.pyplot as plt
import neural_structured_learning as nsl
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## Hiperparámetros

Recolectamos y explicamos los hiperparámetros (en un objeto `HParams`) para el modelo de entrenamiento y evaluación.

Entrada/salida:

- **`input_shape`**: la forma del tensor de entrada. Cada imagen es de 28 por 28 pixeles con 1 canal.
- **`num_classes`**: hay un total de 10 clases, que corresponden a 10 dígitos, de [0-9].

Arquitectura modelo:

- **`conv_filters`**: una lista de números, cada uno especificando la cantidad de filtros de una capa convolucional.
- **`kernel_size`**: el tamaño de la ventana de convolución 2D, compartido por todas las capas convolucionales.
- **`pool_size`**: factores para reducir la imagen a escala en cada capa de agrupación máxima (max-pooling).
- **`num_fc_units`**: la cantidad de unidades (es decir, el ancho) de cada capa totalmente conectada.

Entrenamiento y evaluación

- **`batch_size`**: el tamaño del lote utilizado para el entrenamiento y la evaluación.
- **`epochs`**: la cantidad de épocas de entrenamiento

Aprendizaje adversativo:

- **`adv_multiplier`**: el peso de la pérdida adversativa en el objetivo de aprendizaje, relativo a la pérdida etiquetada.
- **`adv_step_size`**: la magnitud de la perturbación adversativa.
- **`adv_grad_norm`**: la norma para medir la magnitud de la perturbación adversativa.


In [ ]:
class HParams(object):
  def __init__(self):
    self.input_shape = [28, 28, 1]
    self.num_classes = 10
    self.conv_filters = [32, 64, 64]
    self.kernel_size = (3, 3)
    self.pool_size = (2, 2)
    self.num_fc_units = [64]
    self.batch_size = 32
    self.epochs = 5
    self.adv_multiplier = 0.2
    self.adv_step_size = 0.2
    self.adv_grad_norm = 'infinity'

HPARAMS = HParams()

## Conjunto de datos MNIST

El [conjunto de datos MNIST](http://yann.lecun.com/exdb/mnist/) contiene imágenes en escala de grises de números de un dígito escritos a mano (de '0' a '9'). Cada imagen muestra un dígito a baja resolución (28 por 28 pixeles). La tarea consiste en clasificar imágenes en 10 categorías, una por cada dígito.

Aquí cargamos el conjunto de datos MNIST a partir de [conjuntos de datos de TensorFlow](https://www.tensorflow.org/datasets). Incluye la descarga de los datos y la construcción de un `tf.data.Dataset`. El conjunto de datos cargados tiene dos subconjuntos:

- `train` con 60 000 ejemplos y
- `test` con 10 000 ejemplos.

Los ejemplos de ambos subconjuntos están almacenados en los diccionarios de características con las siguientes dos claves:

- `image`: arreglo de valores de pixeles, que van de 0 a 255.
- `label`: etiqueta de verdad fundamental, de 0 a 9.

In [ ]:
datasets = tfds.load('mnist')

train_dataset = datasets['train']
test_dataset = datasets['test']

IMAGE_INPUT_NAME = 'image'
LABEL_INPUT_NAME = 'label'

Para hacer numéricamente estable al modelo, normalizamos los valores de los pixeles a [0, 1] mapeando el conjunto de datos con la función `normalize`. Después de aleatorizar el conjunto de entrenamiento y de agruparlo en lotes, convertimos los ejemplos en tuplas `(image, label)` para entrenamiento del modelo base. También proporcionamos una función para convertir tuplas en diccionarios, para usarlos más adelante.

In [ ]:
def normalize(features):
  features[IMAGE_INPUT_NAME] = tf.cast(
      features[IMAGE_INPUT_NAME], dtype=tf.float32) / 255.0
  return features

def convert_to_tuples(features):
  return features[IMAGE_INPUT_NAME], features[LABEL_INPUT_NAME]

def convert_to_dictionaries(image, label):
  return {IMAGE_INPUT_NAME: image, LABEL_INPUT_NAME: label}

train_dataset = train_dataset.map(normalize).shuffle(10000).batch(HPARAMS.batch_size).map(convert_to_tuples)
test_dataset = test_dataset.map(normalize).batch(HPARAMS.batch_size).map(convert_to_tuples)

## Modelo base

Nuestro modelo base será una red neuronal compuesta por 3 capas convolucionales seguida por 2 capas totalmente conectadas (tal como se define en `HPARAMS`). Aquí lo definimos con la API funcional Keras. Pruebe con total libertad con otras API o arquitecturas de modelos (p. ej., la aplicación de subclases). Tenga en cuenta que el NSL no es compatible con los tres tipos de API Keras.

In [ ]:
def build_base_model(hparams):
  """Builds a model according to the architecture defined in `hparams`."""
  inputs = tf.keras.Input(
      shape=hparams.input_shape, dtype=tf.float32, name=IMAGE_INPUT_NAME)

  x = inputs
  for i, num_filters in enumerate(hparams.conv_filters):
    x = tf.keras.layers.Conv2D(
        num_filters, hparams.kernel_size, activation='relu')(
            x)
    if i < len(hparams.conv_filters) - 1:
      # max pooling between convolutional layers
      x = tf.keras.layers.MaxPooling2D(hparams.pool_size)(x)
  x = tf.keras.layers.Flatten()(x)
  for num_units in hparams.num_fc_units:
    x = tf.keras.layers.Dense(num_units, activation='relu')(x)
  pred = tf.keras.layers.Dense(hparams.num_classes)(x)
  model = tf.keras.Model(inputs=inputs, outputs=pred)
  return model

In [ ]:
base_model = build_base_model(HPARAMS)
base_model.summary()

A continuación, entrenamos y evaluamos el modelo base.

In [ ]:
base_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['acc'])
base_model.fit(train_dataset, epochs=HPARAMS.epochs)

In [ ]:
results = base_model.evaluate(test_dataset)
named_results = dict(zip(base_model.metrics_names, results))
print('\naccuracy:', named_results['acc'])

Podemos observar que el modelo base logra una exactitud del 99 % en el conjunto de datos de prueba. Veremos, a continuación, cuán robusto es en la sección [Robustez bajo perturbaciones adversativas](#scrollTo=HXK9MGG8lBX3).

## Modelo adversativo regularizado

En este caso mostramos cómo incorporar entrenamiento adversativo en un modelo Keras con unas pocas líneas de código, usando el marco de trabajo NSL. El modelo base se encapsula para crear un nuevo `tf.Keras.Model`, cuyo objetivo de entrenamiento incluye la regularización adversativa.

Primero, creamos un objeto <em>config</em> con todos los hiperparámetros relevantes. Lo hacemos con la función ayudante `nsl.configs.make_adv_reg_config`.

In [ ]:
adv_config = nsl.configs.make_adv_reg_config(
    multiplier=HPARAMS.adv_multiplier,
    adv_step_size=HPARAMS.adv_step_size,
    adv_grad_norm=HPARAMS.adv_grad_norm
)

Ahora podemos encapsular un modelo base con `AdversarialRegularization`. Creamos un modelo base nuevo (`base_adv_model`), para que el que ya existía (`base_model`) se pueda usar más adelante para comparar.

El `adv_model` devuelto es un objeto `tf.keras.Model`, cuyo objetivo de entrenamiento incluye un término de regularización para pérdida adversativa. A fin de calcular la pérdida, el modelo debe tener acceso a la información de la etiqueta (característica `label`), además de la entrada regular (característica `image`). Por este motivo, convertimos los ejemplos de las bases de datos de tuplas, de vuelta a diccionarios. Y le decimos al modelo qué característica contiene la información de la etiqueta a través del parámetro `label_keys`.

In [ ]:
base_adv_model = build_base_model(HPARAMS)
adv_model = nsl.keras.AdversarialRegularization(
    base_adv_model,
    label_keys=[LABEL_INPUT_NAME],
    adv_config=adv_config
)

train_set_for_adv_model = train_dataset.map(convert_to_dictionaries)
test_set_for_adv_model = test_dataset.map(convert_to_dictionaries)

Después, compilamos, entrenamos y evaluamos al modelo adversativo regularizado. Probablemente aparezcan advertencias como "Falta la salida del diccionario de pérdida", que está bien, porque el `adv_model` no depende de la implementación base para calcular la pérdida total.

In [ ]:
adv_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['acc'])
adv_model.fit(train_set_for_adv_model, epochs=HPARAMS.epochs)

In [ ]:
results = adv_model.evaluate(test_set_for_adv_model)
named_results = dict(zip(adv_model.metrics_names, results))
print('\naccuracy:', named_results['sparse_categorical_accuracy'])

Podemos observar que el modelo regularizado adversativo también se desempeña correctamente (con exactitud del 99 %) en el conjunto de prueba.

## Robustez y perturbaciones adversativas

Ahora comparamos el modelo base y el modelo adversativo regularizado en cuanto a robustez bajo perturbaciones adversativas.

Usaremos la función `AdversarialRegularization.perturb_on_batch` para generar ejemplos adversativamente perturbados. Y queremos que la generación parta del modelo base. Para lograrlo, encapsulamos el modelo base con `AdversarialRegularization`. Tenga en cuenta que mientras no invoquemos el entrenamiento (`Model.fit`), las variables aprendidas no cambiarán y el modelo seguirá siendo el mismo de la sección [Modelo base](#scrollTo=JrrMpPNmpCKK).

In [ ]:
reference_model = nsl.keras.AdversarialRegularization(
    base_model, label_keys=[LABEL_INPUT_NAME], adv_config=adv_config)
reference_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['acc'])

En un diccionario, recolectamos los modelos a evaluar y también creamos un objeto de métrica para cada uno de ellos.

Observe que tomamos el `adv_model.base_model` para tener el mismo formato de entrada (que no requiere información de las etiquetas) que el modelo base. Las variables aprendidas en `adv_model.base_model` son iguales a las de `adv_model`.

In [ ]:
models_to_eval = {
    'base': base_model,
    'adv-regularized': adv_model.base_model
}
metrics = {
    name: tf.keras.metrics.SparseCategoricalAccuracy()
    for name in models_to_eval.keys()
}

Aquí mostramos el ciclo para generar ejemplos perturbados y para evaluar modelos con ellos. Guardamos las imágenes, etiquetas y predicciones perturbadas para visualizarlas en la siguiente sección.

In [ ]:
perturbed_images, labels, predictions = [], [], []

for batch in test_set_for_adv_model:
  perturbed_batch = reference_model.perturb_on_batch(batch)
  # Clipping makes perturbed examples have the same range as regular ones.
  perturbed_batch[IMAGE_INPUT_NAME] = tf.clip_by_value(
      perturbed_batch[IMAGE_INPUT_NAME], 0.0, 1.0)
  y_true = perturbed_batch.pop(LABEL_INPUT_NAME)
  perturbed_images.append(perturbed_batch[IMAGE_INPUT_NAME].numpy())
  labels.append(y_true.numpy())
  predictions.append({})
  for name, model in models_to_eval.items():
    y_pred = model(perturbed_batch)
    metrics[name](y_true, y_pred)
    predictions[-1][name] = tf.argmax(y_pred, axis=-1).numpy()

for name, metric in metrics.items():
  print('%s model accuracy: %f' % (name, metric.result().numpy()))

Podemos apreciar que la exactitud del modelo base cae drásticamente (de 99 % a alrededor del 50 %) cuando la entrada es perturbada adversativamente. Por otra parte, la exactitud del modelo adversativo regularizado solamente se degrada un poco (del 99 % al 95 %). Es lo que demuestra la efectividad del aprendizaje adversativo en la mejora de la robustez del modelo.

## Ejemplos de imágenes perturbadas adversativamente

Echemos un vistazo a las imágenes perturbadas adversativamente. Podemos observar que las imágenes perturbadas todavía muestran dígitos reconocibles por humanos, pero que pueden engañar al modelo base.

In [ ]:
batch_index = 0

batch_image = perturbed_images[batch_index]
batch_label = labels[batch_index]
batch_pred = predictions[batch_index]

batch_size = HPARAMS.batch_size
n_col = 4
n_row = (batch_size + n_col - 1) // n_col

print('accuracy in batch %d:' % batch_index)
for name, pred in batch_pred.items():
  print('%s model: %d / %d' % (name, np.sum(batch_label == pred), batch_size))

plt.figure(figsize=(15, 15))
for i, (image, y) in enumerate(zip(batch_image, batch_label)):
  y_base = batch_pred['base'][i]
  y_adv = batch_pred['adv-regularized'][i]
  plt.subplot(n_row, n_col, i+1)
  plt.title('true: %d, base: %d, adv: %d' % (y, y_base, y_adv))
  plt.imshow(tf.keras.utils.array_to_img(image), cmap='gray')
  plt.axis('off')

plt.show()

## Conclusión

Hemos demostrado el uso del aprendizaje adversativo para la clasificación de imágenes con el marco de trabajo de aprendizaje estructurado neuronal (NSL). Recomendamos que los usuarios experimenten con diferentes configuraciones adversativas (en hiperparámetros) y que vean cómo estas afectan a la robustez del modelo.